In [1]:
%matplotlib inline
import os
import re
import json, codecs
import pandas as pd
import numpy as np
from dateutil.parser import parse
from datetime import datetime, date, time, timedelta
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import scale
from sklearn import (manifold, decomposition, ensemble, discriminant_analysis, random_projection)
from dateutil.parser import parse
# from sklearn import cross_validation, metrics
# from sklearn.grid_search import GridSearchCV
# from sklearn import model_selection

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [3]:
os.chdir('/project/')
sam = pd.read_csv('./Data/cli_cma_joined_1pct.csv', dtype=object)
# sam_cma = pd.read_csv('./Data/cli_cma_1pct.csv', dtype=object)
# sam_all = pd.merge(sam, sam_cma, on=['CREDITCYCLEFACTMKEY'])
sam.shape

(195343, 251)

In [4]:
sam_sum = sam.describe()
# print_full(sam_sum.transpose())

In [ ]:
# all_col = sam.columns.str.upper()

In [5]:
# import math
# nan_col_ind = [math.isnan(val) if isinstance(val, float) else False for val in sam_sum.ix['top',]]
nan_col_ind = (sam_sum.ix['count',] == 0)
nan_col = sam.columns[nan_col_ind]
nan_col
# sam.drop(nan_col, axis=1, inplace=True)

Index([u'B5SCORE', u'B5RISKREASON1', u'B5RISKREASON2', u'B5RISKREASON3',
       u'B5RISKREASON4', u'VANTAGESCORE', u'VANTAGEREASON1', u'VANTAGEREASON2',
       u'VANTAGEREASON3', u'VANTAGEREASON4', u'OFACFLAG'],
      dtype='object')

In [6]:
oneval_col_ind = (sam_sum.ix['unique',] == 1)
oneval_col = sam.columns[oneval_col_ind]
oneval_col

Index([u'SNAPDATE', u'BRAND', u'LEGACYDAYSDELINQUENT', u'MOSTRECENTRECORDIND',
       u'CTCDCHARGEOFFSAMOUNT', u'LTCDCHARGEOFFSAMOUNT', u'MODELNAME'],
      dtype='object')

In [7]:
# import re
# [re.match('.*date.*', col, re.IGNORECASE) for col in all_col]
# matches = all_col.str.findall('.*date.*', re.IGNORECASE)
# matches.str.get(0).tolist()
date_col_ind = sam.columns.str.contains('DATE')
date_col = sam.columns[date_col_ind]
date_col

Index([u'SNAPDATE', u'CYCLEDATE', u'FIRSTMEASURABLEDATE', u'MAXREVIEWDATE',
       u'INCREASEDATE', u'FIRSTSTATEMENTDATE', u'FIRSTBSBCREDITORDERDATE',
       u'ACCOUNTOPENDATE', u'ACCOUNTCLOSEDATE', u'CHARGEOFFDATE',
       u'CLIINELIGIBLEDATE', u'LASTCOLLECTIONSCASEOPENDATE',
       u'LASTPAYMENTDATE', u'PAYMENTDUEDATE', u'REVIEWDATE',
       u'LASTBANKRUPTCYDATE'],
      dtype='object')

In [8]:
date_col2 = date_col.tolist()
date_col2.remove('CYCLEDATE')
date_col2

['SNAPDATE',
 'FIRSTMEASURABLEDATE',
 'MAXREVIEWDATE',
 'INCREASEDATE',
 'FIRSTSTATEMENTDATE',
 'FIRSTBSBCREDITORDERDATE',
 'ACCOUNTOPENDATE',
 'ACCOUNTCLOSEDATE',
 'CHARGEOFFDATE',
 'CLIINELIGIBLEDATE',
 'LASTCOLLECTIONSCASEOPENDATE',
 'LASTPAYMENTDATE',
 'PAYMENTDUEDATE',
 'REVIEWDATE',
 'LASTBANKRUPTCYDATE']

In [9]:
to_drop = nan_col.tolist() + oneval_col.tolist() + date_col2
sam2 = sam.drop(to_drop, axis=1)
sam2.shape

(195343, 219)

In [10]:
sam2['INCRTYPE'] = sam2['INCRTYPE'].fillna('Unknown')

In [ ]:
sam2['CURRENTDAYSPASTDUE'].value_counts()

In [11]:
indata = sam2
out_data = pd.DataFrame([])
columns_in = sam2.columns.tolist()
columns_excl = ['CYCLEDATE', 'CUSTOMERSEGMENT', 'DWACCTID', 'CUSTOMERNUMBER', 'RANDOMNUM', 'CREDITCYCLEFACTMKEY']
all_cat = []
all_con = []
for col_name in columns_in:
    print col_name
    col_out = indata[col_name]
    if col_name not in columns_excl:
        if (sam_sum.ix['unique',col_name] > 20):
            out_data[col_name] = pd.to_numeric(col_out)
            all_con.append(col_name)
        if (sam_sum.ix['unique',col_name] <= 20):
            out_data[col_name] = pd.Series(pd.Categorical([v for v in col_out],categories=col_out.unique()))
            all_cat.append(col_name)
    else: out_data[col_name] = col_out

DWACCTID
CUSTOMERNUMBER
CYCLEDATE
HADINCREASE
INCRTYPE
RANDOMNUM
CREDITCYCLEFACTMKEY
SCORE01
SCORE02
SCORE03
SCORE04
SCORE05
SCORE06
SCORE07
SCORE08
SCORE09
STATEMENTMINIMUMPAYMENTDUE
ADDRESSBADIND
CURRENTBAL
ACCUMDEFERREDINTEREST
CLIINELIGIBLEIND
COLLECTIONCODE
CREDITLIMITAMOUNT
CREDITLIMITCHANGEREASON
CURRENTDAYSPASTDUE


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:15: FutureWarning: 
Setting NaNs in `categories` is deprecated and will be removed in a future version of pandas.


CURRENTDAYSPASTDUEDELINQUENT
CUSTOMERAPR
CUSTOMERSEGMENT
CYCLEDUE
DAYSDELQ
DAYSSINCELASTSALE
DECEASEDCODE
DTIFLAG
EMAILBADIND
EXTERNALSTATUSCODE
EXTERNALSTATUSREASONCODE
FINANCECHARGEBAL
HOMEPHONEBADIND
INTERNALSTATUSCODE
LTDMAXDAYSDELQ
LTDNETPAYMENTTALLY
MANUALSTATUSCODE
MARGINRATE
NONDEFERREDBAL
NOPAYDEFERREDBAL
NSFCODE
NUMBEROFMTHSINACTIVE
OTHERFEEBAL
OVERLIMITAMOUNT
PLACEDORDERFLAG
CTCDPAYMENTAMOUNT
CTCDPAYMENTRVSLAMOUNT
CTCDPAYMENTTALLY
CTCDBSBPURCHAMOUNT
CTCDBSBPURCHRVSLAMOUNT
CTCDBSBPURCHRVSLTALLY
CTCDBSBPURCHTALLY
CTCDNSFFEEAMOUNT
CTCDLATECHARGEAMOUNT
CTCDLATECHARGERVSLAMOUNT
LTCDPAYMENTAMOUNT
LTCDPAYMENTRVSLAMOUNT
LTCDPAYMENTTALLY
LTCDBSBPURCHAMOUNT
LTCDBSBPURCHRVSLAMOUNT
LTCDBSBPURCHRVSLTALLY
LTCDBSBPURCHTALLY
FHACCOUNTNUMBER
SCORE
FRAUDALERTFLAG
ADDRESSDISCREPANCYFLAG
FRAUDALERTCODE
CRITERIACODEANDDNSCODE
RISKREASON1
RISKREASON2
RISKREASON3
RISKREASON4
OPENITYPETRADEBALTOTAL
MO12BANKINSTALLCOUNT
OPENBANKRTYPEHIGHCREDITTOTAL
GOODRTYPETRADECOUNT
MO12REVOPENBANKCARDCOUNT
R6039_

In [52]:
CYCLEYD = [x[:7] for x in out_data['CYCLEDATE']]
out_data['CYCLEYD'] = CYCLEYD

In [54]:
(out_data[['SCORE01', 'SCORE02', 'CYCLEYD', 'DWACCTID']]).set_index(['CYCLEYD', 'DWACCTID']).unstack(0)

SCORE01                                                          \
CYCLEYD  2012-09 2012-10 2012-11 2012-12 2013-01 2013-02 2013-03 2013-04   
DWACCTID                                                                   
10000157     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10001045     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10001834     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10002499     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10003697     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10004662     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10005291     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10010579     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10016062     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10017998     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10021486     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10021671     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10023286     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10028347     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10029064     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10029662     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1003064     -1.0    -1.0    -1.0    -1.0    -1.0    -1.0    -1.0    -1.0   
10032837     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10036045     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10038415     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10042831     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10046768     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10047001     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10048101     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10049970     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10051171     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10052777     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10053527     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10053942     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10056774     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
...          ...     ...     ...     ...     ...     ...     ...     ...   
9941178      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9941522      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9941738      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9942030      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9954078      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9955413      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
996365      -1.0    -1.0    -1.0    -1.0    -1.0    -1.0    -1.0    -1.0   
9966333      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9967673      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9968199      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9968646      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9972881      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9974217      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9974347      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9976850      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9977870      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9978091      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9978734      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9980671      NaN     NaN     NaN     NaN     NaN     NaN 

In [12]:
out_data['CUSTOMERSEGMENT'] = pd.Series(pd.Categorical([v for v in out_data['CUSTOMERSEGMENT']],categories=out_data['CUSTOMERSEGMENT'].unique()))

In [ ]:
out_data.pivot_table('CURRENTBAL', index=['CUSTOMERSEGMENT', 'INCRTYPE'], columns='HADINCREASE').plot(kind='barh')

In [57]:
r_scores = out_data.filter(regex=('R[0-9]{4}_.*'))
r_scores['CUSTOMERNUMBER'] = out_data['CUSTOMERNUMBER']
cus_max = r_scores.groupby('CUSTOMERNUMBER').max()
overall_max = cus_max.max()
overall_max

R6039_OPENBANKREVTRADECOUNT                      99.0
R6051_MOSINCEDLAFORBANKREVTRADE                9999.0
R6054_REVTRADEPASTDUECOUNT                       99.0
R6063_REVTRADESATISFACTORYCOUNT                  99.0
R6093_MO24REVTRADENEVER30DCOUNT                  99.0
R6114_REVTRADE75PERHIGHCOUNT                     99.0
R6117_REVTRADE50PERHIGHCOUNT                     99.0
R6195_AGE                                        99.0
R6196_BALHIGHCREDITPERCENT                    99999.0
R6197_BANKBALHIGHCREDITPERCENT                99999.0
R6198_DEPTBALHIGHCREDITDEPTPERCENT            99999.0
R6199_CONSUMERBALHIGHCREDITPERCENT            99999.0
R6200_SATISFACTORYTRADEPERCENT                99999.0
R6205_MO24TRADE90DCOUNT                          99.0
R6206_MO24BANKTRADE90DCOUNT                      99.0
R7006_MO24PUBLICRECNOMEDCOUNT                    99.0
R8003_PUBLICRECPLUSBADDEBTNOMEDCOUNT             99.0
R8004_MO24BADDEBTPLUSPUBLICRECDEROGCOUNT         99.0
R6246_TRADE30DORWORSECOUNT  

In [59]:
cma_scores = out_data.filter(regex=('CMA[0-9]{4}'))
cma_scores['CUSTOMERNUMBER'] = out_data['CUSTOMERNUMBER']
cus_max = cma_scores.groupby('CUSTOMERNUMBER').max()
overall_max = cus_max.max()
overall_max

CMA3000         99.0
CMA3001         99.0
CMA3109         99.0
CMA3120       9999.0
CMA3139         99.0
CMA3146         99.0
CMA3159    9999999.0
CMA3161    9999999.0
CMA3164    9999999.0
CMA3165    9999999.0
CMA3168    9999999.0
CMA3204    9999999.0
CMA3208    9999999.0
CMA3235    9999999.0
CMA3236    9999999.0
CMA3907         99.0
CMA3909         99.0
CMA3911    9999999.0
CMA3913    9999999.0
CMA3842      99999.0
CMA3845      99999.0
CMA3854      99999.0
CMA3856      99999.0
CMA3917         99.0
CMA3920         99.0
CMA3926         99.0
CMA3932         99.0
CMA3170    9999999.0
CMA3209    9999999.0
CMA3662         99.0
dtype: float64

In [58]:
scores = out_data.filter(regex=('SCORE[0-9]{2}'))
scores['CUSTOMERNUMBER'] = out_data['CUSTOMERNUMBER']
cus_min = scores.groupby('CUSTOMERNUMBER').min()
overall_min = cus_min.min()
overall_min

SCORE01   -1
SCORE02   -1
SCORE03   -1
SCORE04   -1
SCORE05   -1
SCORE06   -1
SCORE07   -1
SCORE08   -1
SCORE09   -1
dtype: int64

In [ ]:
x = np.array(['min', 'max', 'last'])
l = len(all_con)
con_op = dict(zip(all_con, np.repeat(x[np.newaxis,:], l, 0)))
con_op['CMA3208']

In [ ]:
con_ft_raw = out_data[all_con + ['CUSTOMERNUMBER']]
con_ft = con_ft_raw.groupby('CUSTOMERNUMBER').agg(con_op)

In [ ]:
con_ft.columns = ['_'.join(col).strip() for col in con_ft.columns.values]

In [ ]:
pd.Series([1,2,3]).pct_change()

In [15]:
out = out_data.sort_values(['CUSTOMERNUMBER', 'CYCLEDATE']).groupby(['CUSTOMERNUMBER']).last().reset_index()

In [13]:
con_ft_raw = out_data[all_con + ['CUSTOMERNUMBER']]
min_ft = con_ft_raw.groupby('CUSTOMERNUMBER').apply(lambda x:x.min())
max_ft = con_ft_raw.groupby('CUSTOMERNUMBER').apply(lambda x:x.max())
con_ft = pd.merge(min_ft, max_ft, on='CUSTOMERNUMBER', suffixes=('_min', '_max'))

In [16]:
last_ft = out.drop(['DWACCTID', 'CYCLEDATE', 'RANDOMNUM', 'CREDITCYCLEFACTMKEY', 'CYCLEDUE'], axis=1)
last_ft['CUSTOMERNUMBER'] = last_ft['CUSTOMERNUMBER'].map(float)
all_ft = pd.merge(con_ft, last_ft, on='CUSTOMERNUMBER', suffixes=('', '_last'))

In [17]:
target_raw = out_data[['CUSTOMERNUMBER', 'CYCLEDUE']]
target = target_raw.groupby('CUSTOMERNUMBER').max()

In [18]:
dmat = pd.get_dummies(all_ft, prefix_sep='__')

In [ ]:
out_data.columns.tolist()

In [ ]:
# dmat.drop(['CUSTOMERNUMBER'], axis=1, inplace=True)
# target.drop(['CUSTOMERNUMBER'], axis=1, inplace=True)

In [19]:
xgb_model = xgb.XGBRegressor().fit(dmat, target)

In [20]:
wt = xgb_model.booster().get_score(importance_type='weight')

In [21]:
wt2 = pd.DataFrame(wt.items(), columns=['Feature', 'Weight'])
print_full(wt2.sort_values(['Weight'], ascending=False))

                                      Feature  Weight
108          CURRENTDAYSPASTDUEDELINQUENT_max     183
21                      DAYSSINCELASTSALE_max      37
107                               SCORE06_max      33
37                         LTDMAXDAYSDELQ_max      27
39                                SCORE01_min      19
82                   CTCDLATECHARGEAMOUNT_max      17
64                         NONDEFERREDBAL_max      16
26                            OTHERFEEBAL_max      15
34                   NUMBEROFMTHSINACTIVE_max      12
51                        OVERLIMITAMOUNT_max      11
50                                SCORE07_max      11
77                                SCORE02_max      10
74                        FHACCOUNTNUMBER_min      10
47                                CMA3856_max       9
73                          DAYSSINCELASTSALE       9
71                     LTDNETPAYMENTTALLY_max       9
93                       FINANCECHARGEBAL_max       9
13                 STATEMENT

In [22]:
gavin = wt2['Feature'].str.split('_').map(lambda x:x[0]).unique()
gavin_derived = wt2['Feature']

In [23]:
peng_ltw = [
'SCORE01', 
'SCORE05', 
'NOPAYDEFERREDBAL', 
'R6196_BALHIGHCREDITPERCENT', 
'R6011_OPENTRADEBALTOTAL', 
'CTCDBSBPURCHAMOUNT', 
'CYCLEDUE'
]
peng_latest = [
"SCORE04","SCORE06","SCORE07","SCORE08","SCORE09","OPENMORTGAGETRADECOUNT","R6008_MO24TRADEOPENCOUNT",
"FRAUDALERTCODE","OPENBANKRTYPEHIGHCREDITTOTAL","R6015_CURRENTPASTDUEAMOUNT","R6035_OPENTRADEBAL50PERHIGHCOUNT",
"NONFHBALTOTAL","R6027_MO24TRADENEVER30DCOUNT","OTHERFEEBAL","LTDNETPAYMENTTALLY","DONOTSOLICITFLAG",
"DAYSSINCELASTSALE","EXTERNALSTATUSREASONCODE","FINANCECHARGEBAL","CTCDLATECHARGERVSLAMOUNT","SCOREINDICATOR",
"RMS5002","OVERLIMITAMOUNT","LTCDBSBPURCHRVSLTALLY","BANKRUPCYFLAG","ALLTRADEPAYMENTTOTAL",
"R7000_DEROGATORYPUBLICRECORDCOUNT","MO12TRADE30DLATECOUNT","CTCDLATECHARGEAMOUNT","CTCDBSBPURCHTALLY",
"R6009_TRADECOUNT","R6115_DEPTTRADEBAL75HIGHCREDITCOUNT","R5006_MO06PROMOTIONINQUIRYCOUNT","MOSINCEMAJORDEROG",
"R6021_BADDEBTTRADECOUNT","LTCDPAYMENTRVSLAMOUNT","MARGINRATE","CTCDPAYMENTAMOUNT","CREDITLIMITAMOUNT","CURRENTBAL",
"CURRENTDAYSPASTDUEDELINQUENT","CTCDPAYMENTTALLY","R6042_TOTALOPENBALANCE","R6040_OPENDEPTTRADECOUNT",
"CTCDPAYMENTRVSLAMOUNT","CRITERIACODEANDDNSCODE","NUMBEROFMTHSINACTIVE","NONFHOPENTRADECOUNT","R6010_OPENTRADECOUNT",
"R6018_D30TRADECOUNT","R5003_MO06PROMOTIONINQUIRYCOUNT2","R6003_OLDESTTRADEAGE","ACCUMDEFERREDINTEREST","CUSTOMERAPR",
"CTCDNSFFEEAMOUNT","ADDRESSDISCREPANCYFLAG","BRANDXID","R6117_REVTRADE50PERHIGHCOUNT","R8000_COLLECTIONCOUNT",
"CUSTOMERSEGMENT","R5001_MOSINCELASTINQUIRYCOUNT","HOMEPHONEBADIND","R6073_D90DEPTTRADECOUNT","R6066_D30REVTRADECOUNT",
"SCORE","LTCDBSBPURCHRVSLAMOUNT","MO12OPENRETAILCOUNT","MO03TRADECLOSEDCOUNT","ADDRESSBADIND","GOODRTYPETRADECOUNT",
"R6199_CONSUMERBALHIGHCREDITPERCENT","R8003_PUBLICRECPLUSBADDEBTNOMEDCOUNT","MOSINCELASTDELINQUENCYCOUNT",
"MO03PROMOTIONINQUIRYCOUNT","LTCDPAYMENTAMOUNT","R8002_MOSINCELASTACTIVITYCOUNT","R6195_AGE",
"R6054_REVTRADEPASTDUECOUNT","LTDMAXDAYSDELQ","R6022_TRADENEVER30DCOUNT","PLACEDORDERFLAG","MO12BANKINSTALLCOUNT",
"R5007_MO06BANKPROMOTIONINQUIRYCOUNT"
]

In [55]:
peng_all = peng_ltw + peng_latest
all = set(gavin_derived) | set(peng_all)
gavin_only = set(all) - set(peng_all)
list(gavin_only)

['R6246_TRADE30DORWORSECOUNT_min',
 'R7006_MO24PUBLICRECNOMEDCOUNT_max',
 'RISKREASON4_min',
 'SCORE03',
 'SCORE02',
 'CMA3235_min',
 'CMA3109_max',
 'DAYSSINCELASTSALE_min',
 'R6063_REVTRADESATISFACTORYCOUNT_max',
 'STATEMENTMINIMUMPAYMENTDUE',
 'R6012_OPENTRADEHIGHCREDITTOTAL',
 'R6022_TRADENEVER30DCOUNT_min',
 'LTCDPAYMENTAMOUNT_max',
 'CTCDLATECHARGERVSLAMOUNT_min',
 'RISKREASON3_max',
 'R6011_OPENTRADEBALTOTAL_max',
 'DAYSSINCELASTSALE_max',
 'CMA3168_max',
 'LTCDPAYMENTAMOUNT_min',
 'OTHERFEEBAL_max',
 'CTCDBSBPURCHAMOUNT_max',
 'CMA3235',
 'CMA3236',
 'ALLTRADEPAYMENTTOTAL_min',
 'CUSTOMERSEGMENT__A ',
 'CTCDPAYMENTAMOUNT_min',
 'NUMBEROFMTHSINACTIVE_max',
 'CMA3845_max',
 'R6196_BALHIGHCREDITPERCENT_max',
 'LTDMAXDAYSDELQ_max',
 'SCORE01_min',
 'LTDMAXDAYSDELQ_min',
 'R6004_NEWESTTRADEAGE_max',
 'R6010_OPENTRADECOUNT_max',
 'R6040_OPENDEPTTRADECOUNT__2.0',
 'CMA3204',
 'R6010_OPENTRADECOUNT_min',
 'CMA3856_min',
 'CMA3856_max',
 'R6063_REVTRADESATISFACTORYCOUNT_min',
 'SCORE07_

In [56]:
gavin

array(['SCORE05', 'R7006', 'R6027', 'RISKREASON4', 'SCORE01', 'SCORE03',
       'SCORE02', 'CMA3235', 'R6200', 'DAYSSINCELASTSALE', 'R6063',
       'RMS5002', 'STATEMENTMINIMUMPAYMENTDUE', 'R6012', 'R6022',
       'LTCDPAYMENTAMOUNT', 'R6196', 'CTCDLATECHARGERVSLAMOUNT',
       'CTCDLATECHARGEAMOUNT', 'R6011', 'CMA3168', 'R5006', 'R6195',
       'OTHERFEEBAL', 'CTCDBSBPURCHAMOUNT', 'NONFHBALTOTAL', 'CURRENTBAL',
       'ALLTRADEPAYMENTTOTAL', 'CUSTOMERSEGMENT', 'CTCDPAYMENTAMOUNT',
       'NUMBEROFMTHSINACTIVE', 'CMA3845', 'LTDMAXDAYSDELQ', 'R6004',
       'R6010', 'R6040', 'CMA3204', 'CMA3856', 'SCORE07',
       'OVERLIMITAMOUNT', 'SCORE', 'CMA3842', 'SCORE04', 'R6034',
       'MO12TRADE60DLATECOUNT', 'R5001', 'NOPAYDEFERREDBAL', 'CMA3662',
       'NONDEFERREDBAL', 'R6093', 'CMA3854', 'CMA3911', 'CMA3109',
       'LTDNETPAYMENTTALLY', 'R6015', 'FHACCOUNTNUMBER',
       'LTCDPAYMENTTALLY', 'OPENBANKRTYPEHIGHCREDITTOTAL', 'CMA3161',
       'LTCDBSBPURCHAMOUNT', 'CMA3907', 'R5003', 'R624

In [ ]:
all_all = set(gavin_derived) | set(all)

In [ ]:
one_cust_id = sam2['DWACCTID'].sample(1)
one_cust = sam2.ix[sam2['DWACCTID'].isin(one_cust_id),:]
print_full(one_cust)